In [13]:
import pickle
import pandas as pd
import datetime
import unirest
from glob import glob
from collections import Counter
from get_tweets import get_tweets

In [2]:
with open('movies_data1.pkl', 'r') as picklefile:
    movies_scraped, movies_skipped = pickle.load(picklefile)

In [3]:
movies = pd.DataFrame(movies_scraped)
movies.dropna(axis=0, subset=['domestic_total_gross'], inplace=True)
movies.sort_values(by='domestic_total_gross', ascending=False, inplace=True)

In [39]:
print len(movies)
print movies[:10]

14682
              BOM_id                                             actors  \
9982       starwars7  John BoyegaDaisy RidleyAdam DriverOscar IsaacA...   
13275         avatar  Sam WorthingtonZoe SaldanaSigourney WeaverMich...   
1772   jurassicpark4  Nick RobinsonOmar SyChris PrattBryce Dallas Ho...   
2616      avengers11  Robert Downey, Jr.Chris HemsworthChris EvansJe...   
7211         titanic  Leonardo DiCaprioKate WinsletBilly ZaneKathy B...   
9173      darkknight  Christian BaleHeath LedgerAaron EckhartMorgan ...   
4547       pixar2015  Ellen DeGeneres (Voice)Albert Brooks (Voice)Ed...   
3565       avengers2  Robert Downey, Jr.Chris HemsworthMark RuffaloC...   
953          batman3  Christian BaleMichael CaineAnne HathawayTom Ha...   
4183          shrek2  Mike Myers (Voice)Cameron Diaz (Voice)Eddie Mu...   

            budget                                   director   distributor  \
9982   245000000.0                                J.J. Abrams   Buena Vista   
13275     

In [22]:
print movies['genre'].unique()
print movies['genre'].value_counts()

[u'Sci-Fi Fantasy' u'Sci-Fi Adventure' u'Action / Adventure' u'Romance'
 u'Animation' u'Action Thriller' u'Period Adventure' u'Sci-Fi Action'
 u'Fantasy' u'Historical Drama' u'Adventure' u'Action' u'Family Adventure'
 u'Sci-Fi Horror' u'Drama' u'Comedy / Drama' u'Horror' u'Family Comedy'
 u'Comedy' u'Sci-Fi Thriller' u'Horror Thriller' u'Sports Drama'
 u'Sci-Fi Comedy' u'Fantasy Comedy' u'Action Drama' u'Romantic Comedy'
 u'Action Comedy' u'Horror Comedy' u'Sci-Fi' u'Fantasy Drama' u'Thriller'
 u'War' u'Period Action' u'Action Horror' u'Historical Epic' u'Western'
 u'Crime Comedy' u'Adventure Comedy' u'Period Drama' u'Musical'
 u'Sports Comedy' u'Drama / Thriller' u'Crime Drama' u'Foreign / Action'
 u'Period Horror' u'Music Drama' u'Western Comedy' u'Documentary'
 u'War Drama' u'Sports Action' u'Period Comedy' u'Crime' u'Action / Crime'
 u'War Romance' u'IMAX' u'Action Fantasy' u'Crime Thriller'
 u'Romantic Thriller' u'Comedy Thriller' u'Family' u'Romantic Adventure'
 u'Concert' u'Fore

In [4]:
time_range = datetime.timedelta(days=45)
maxtweets = 1000

def fetch_tweets(row):
    BOM_id = row['BOM_id']
    movie_title = row['movie_title']
    query = BOM_id + ' OR #' + BOM_id + ' OR ' + movie_title + ' OR #' + movie_title
    date = row['release_date']
    start = date.date().strftime('%Y-%m-%d')
    end = (date.date() + time_range).strftime('%Y-%m-%d')
    get_tweets(filename='data/' + BOM_id, maxtweets=maxtweets, query=query, since=start, until=end)
    print movie_title, start, end

In [5]:
# for _, row in movies[:5].iterrows():
#     fetch_tweets(row)
    
for _, row in movies[-2005:-2000].iterrows():
    fetch_tweets(row)

Done. Output file generated "data/somebodytolove".
Somebody to Love 1996-09-27 1996-11-11
Done. Output file generated "data/womenintrouble".
Women in Trouble 2009-11-13 2009-12-28
Done. Output file generated "data/dare09".
Dare 2009-11-13 2009-12-28
Done. Output file generated "data/toscaskiss".
Tosca's Kiss 2014-06-06 2014-07-21
Done. Output file generated "data/totalrecall2012rerelease".
Total Recall 2012-08-10 2012-09-24


In [38]:
fetched_movie = {}
directories = glob('data/*')

for directory in directories:
    BOM_id = directory[5:]
    print BOM_id
    fetched_movie[BOM_id] = pd.read_csv(directory, sep=';')
    
# for BOM_id in movies['BOM_id'][:5]:
#     print BOM_id
#     fetched_movie[BOM_id] = pd.read_csv('data/' + BOM_id, sep=';')

# for BOM_id in movies['BOM_id'][-5005:-5000]:
#     print BOM_id
#     fetched_movie[BOM_id] = pd.read_csv('data/' + BOM_id, sep=';')

avatar
avengers11
jurassicpark4
misterlonely
starwars7
totalrecall2012rerelease
womenintrouble


In [36]:
def sentiment(text):
    # response = requests.post('http://text-processing.com/api/sentiment/', data={'text': text})
    response = unirest.post("https://japerk-text-processing.p.mashape.com/sentiment/",
        headers={
            "X-Mashape-Key": "EMrBfg9GO4mshgbevq2BtBZCdet3p1iXIWejsnKRDuWRNljYxI",
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json"
          },
        params={
            "language": "english",
            "text": text
          }
    )
    if response.code != 200:
        print response.code
        print response.body
        print 'failed: {}'.format(text)
        return {'sentiment': None, 'probability': None}
    sentiment = response.body['label']
    if sentiment == 'pos':
        return {'sentiment': 1, 'probability': response.body['probability']}
    elif sentiment == 'neg':
        return {'sentiment': -1, 'probability': response.body['probability']}
    else:
        return {'sentiment': 0, 'probability': response.body['probability']}

In [32]:
pd.DataFrame([sentiment('&#039; Star Wars: The Force Awakens &#039; Box Office: Nearing $900 Million US And $2 Billion Worldwide - Forbes http:// dragplus.com/post/id/334382 04')])

400
Form Validation Errors
text: This field is required.

failed: &#039; Star Wars: The Force Awakens &#039; Box Office: Nearing $900 Million US And $2 Billion Worldwide - Forbes http:// dragplus.com/post/id/334382 04


,probability,sentiment
0,None,None


In [39]:
for BOM_id, movie in fetched_movie.items():
    sentiments = pd.DataFrame([sentiment(text) for text in movie['text']])
    fetched_movie[BOM_id] = pd.concat([movie, sentiments], axis=1)
    #movie.to_csv('data/' + BOM_id, sep=';')
    print BOM_id

400
Form Validation Errors
text: This field is required.

failed: &#039; Star Wars: The Force Awakens &#039; Box Office: Nearing $900 Million US And $2 Billion Worldwide - Forbes http:// dragplus.com/post/id/334382 04 …
400
Form Validation Errors
text: This field is required.

failed: &#039; Star Wars: The Force Awakens &#039; Box Office: Nearing $900 Million U.S. And $2 Billion Worldwide - http:// newsbunch.com/uncategorized/ star-wars-the-force-awakens-box-office-nearing-900-million-u-s-and-2-billion-worldwide/&#039;St … ...
starwars7
avengers11
womenintrouble
misterlonely
totalrecall2012rerelease
avatar
jurassicpark4


In [42]:
for BOM_id, movie in fetched_movie.items():
    movie.to_csv('data/' + BOM_id, sep=';')

In [47]:
for title, movie in fetched_movie.items():
    counter = Counter(movie['sentiment'])
    print title, " : ", counter, 'total: ', sum(counter.values())

starwars7  :  Counter({0.0: 650, 1.0: 335, -1.0: 11, nan: 1, nan: 1}) total:  998
avengers11  :  Counter({1: 777, 0: 183, -1: 39}) total:  999
womenintrouble  :  Counter({0: 370, 1: 192, -1: 113}) total:  675
misterlonely  :  Counter({0: 16, 1: 16, -1: 6}) total:  38
totalrecall2012rerelease  :  Counter({-1: 635, 1: 253, 0: 111}) total:  999
avatar  :  Counter({1: 467, 0: 287, -1: 246}) total:  1000
jurassicpark4  :  Counter({0: 481, 1: 334, -1: 185}) total:  1000
